Motion Correction for multiple subjects in bulk:

In [ ]:
# get subjects from subjects.txt:
subjects_file = r"D:\HCP_vessel_files\subjects.txt"

with open(subjects_file, "r") as f:
    subjects = [line.strip() for line in f if line.strip()]

print("Loaded", len(subjects), "subjects:")
# subjects = subjects[:10] # only do the first 10 subjects:
print(subjects)  # show the first 10 to check


Loaded 190 subjects:
['100307', '101309', '102008', '102311', '103111', '103414', '103818', '105014', '107422', '108828', '109123', '110411', '111716', '113619', '113821', '113922', '114419', '115320', '116524', '118528', '118932', '120212', '121618', '123420', '124220', '124826', '127630', '128127', '129533', '130013', '132118', '133827', '136833', '137027', '137128', '137936', '138231', '140117', '140824', '142424', '142828', '143325', '144832', '145531', '146331', '146432', '148941', '150423', '150726', '151627', '153833', '154734', '154936', '155635', '156637', '158035', '159138', '161630', '161731', '162733', '163331', '164131', '165032', '167743', '168341', '169343', '169444', '170934', '172130', '172938', '173536', '175035', '177645', '178950', '179346', '180129', '180836', '180937', '181232', '183034', '185139', '186141', '187143', '187547', '188347', '189349', '191033', '191437', '191841', '192540', '192843', '194140', '194847', '195849', '197348', '198855', '199150', '199251'

In [2]:
import ants
import os
import numpy as np
from glob import glob
from scipy.io import loadmat
from scipy.spatial.transform import Rotation as R
import shutil
import gzip # to save as compressed nifti 


for subjectid in subjects:
    file_path = fr"D:\hcp_processed\{subjectid}\motion_corrected\{subjectid}_REST1_LR_mc.nii.gz"
    if os.path.exists(file_path):
        print(f"Skipping {subjectid}, file already exists.")
        continue  # Skip this subject
    else:
        print(f"Processing {subjectid}...")

    # print(f"subject {subjectid}")
    # Input and output paths
    input_nifti = fr"D:\hcp_data\{subjectid}\unprocessed\3T\rfMRI_REST1_LR\{subjectid}_3T_rfMRI_REST1_LR.nii.gz"
    output_dir = fr"D:\hcp_processed\{subjectid}\motion_corrected"
    os.makedirs(output_dir, exist_ok=True)
    output_nifti = os.path.join(output_dir, f"{subjectid}_REST1_LR_mc.nii")
    fmri = ants.image_read(input_nifti, dimension=4)
    mc_output = ants.motion_correction(fmri, type_of_transform='Rigid', verbose=True)

    # Save motion-corrected file
    mc_output['motion_corrected'].to_filename(output_nifti)

    # Compress afterward
    with open(output_nifti, 'rb') as f_in:
        with gzip.open(output_nifti + ".gz", 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    # Remove the uncompressed file
    os.remove(output_nifti)

    # Save motion parameters
    motion_params_path = os.path.join(output_dir, "motion_params.csv")


    motion_param_dir = os.path.join(output_dir, "framewise_affines")
    os.makedirs(motion_param_dir, exist_ok=True)

    for i, frame in enumerate(mc_output['motion_parameters']):
        src = frame[0]  # each item is a list like ['path']
        dst = os.path.join(motion_param_dir, f"frame_{i:04d}_GenericAffine.mat")
        shutil.copy(src, dst)
    # CANNOT FIND GOOD DOCUMENTATION ON THIS PART. MAY NEED TO FLIP SOME THINGS IN THE AFFINE 
    # SINCE HCP IS RAS ORIENTATION. https://sourceforge.net/p/advants/discussion/840261/thread/9fbbaab7/
    # BUT SHOULD BE CONSISTENT ACROSS HCP SUBJECTS

    # Path to saved .mat transforms
    affine_dir = fr"{output_dir}\framewise_affines"
    affine_files = sorted(glob(os.path.join(affine_dir, "*.mat")))

    motion_params = []

    for mat_path in affine_files:
        mat_data = loadmat(mat_path)
        affine_vector = mat_data['AffineTransform_float_3_3']  # shape (12, 1)

        # First 9 are the 3x3 rotation matrix
        rot_matrix = affine_vector[:9].reshape((3, 3))

        # Last 3 are translations
        translation = affine_vector[9:].flatten()

        # Convert rotation matrix to Euler angles (in degrees)
        rot = R.from_matrix(rot_matrix)
        euler_deg = rot.as_euler('xyz', degrees=True)

        # Store [Tx, Ty, Tz, Rx, Ry, Rz]
        motion_params.append(np.concatenate([translation, euler_deg]))

    motion_array = np.array(motion_params)
    np.save(fr"{output_dir}\motion_array.npy", motion_array) # save motion array for each subject 



Skipping 100307, file already exists.
Skipping 101309, file already exists.
Skipping 102008, file already exists.
Skipping 102311, file already exists.
Skipping 103111, file already exists.
Skipping 103414, file already exists.
Skipping 103818, file already exists.
Skipping 105014, file already exists.
Skipping 107422, file already exists.
Skipping 108828, file already exists.
Skipping 109123, file already exists.
Skipping 110411, file already exists.
Skipping 111716, file already exists.
Skipping 113619, file already exists.
Skipping 113821, file already exists.
Skipping 113922, file already exists.
Skipping 114419, file already exists.
Skipping 115320, file already exists.
Skipping 116524, file already exists.
Skipping 118528, file already exists.
Skipping 118932, file already exists.
Skipping 120212, file already exists.
Skipping 121618, file already exists.
Skipping 123420, file already exists.
Skipping 124220, file already exists.
Skipping 124826, file already exists.
Skipping 127